## Using Tensorflow to implement walkmate coach action

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np # linear algebra
import seaborn as sns
sns.set(style='whitegrid')
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tf
import random


import math
from tensorflow import keras
from tensorflow.keras import layers


In [ ]:
from subprocess import check_output
print(check_output(["ls", "./input"]).decode("utf8"))

 **Step 1: Read the data**

In [ ]:
walkmate = pd.read_csv('./input/Walkmate_Boost_Sense.csv')

In [ ]:
walkmate.shape

In [ ]:
walkmate.head()

I want to do a multi class classification
Predicting coach action to boost : Motivation vs Ability vs Propensity vs Other vs None

In [ ]:
#sns.pairplot(walkmate[['age', 'gender', 'baseline_steps', 'block_action']], diag_kind='kde')

In [ ]:
walkmate['step_count_prev'].replace(np.nan, 0, inplace=True)
walkmate['step_count_prev_1'].replace(np.nan, 0, inplace=True)
walkmate['step_count_prev_2'].replace(np.nan, 0, inplace=True)
walkmate['step_count_prev_3'].replace(np.nan, 0, inplace=True)
walkmate['step_count_prev_4'].replace(np.nan, 0, inplace=True)
walkmate.isnull().values.any()

In [ ]:
list(walkmate.columns)

**Step 0 : Sub select data for training** 

* Drop not relevant rows 
* Drop irrelevant columns

In [ ]:
walkmate.drop(walkmate[walkmate['coach_type'] == 'ASSISTANT'].index, inplace = True)
walkmate.drop(walkmate[ (walkmate['Attendance'] == 'AS') | (walkmate['Attendance'] == 'A')].index, inplace = True)
walkmate.drop(walkmate[walkmate['block_reward'] == 'Unsuccessful'].index, inplace = True)

In [ ]:
walkmate.shape

**Step 3: Split data based on participant-id** 

* trainset: 80%
* testset: 20%

In [ ]:
# set seed for numpy and tensorflow
# set for reproducible results
seed = 5
test_portion = 0.2
titration_portion = 1.0
np.random.seed(seed)
tf.random.set_seed(seed)

In [ ]:
all_users = list(walkmate[walkmate['coach_type']=='DIRECT']['participant_id'].unique())
print(all_users)
test_users = random.sample(all_users, int(test_portion * len(all_users)))
print(test_users)
walkmate.drop(labels=[ 'block_reward', 'coach_id', 'coach_type', 'version_id', 'conv_turn', 'duration', 'promptness', 'achieved_step_count', 'difference_achieved_baseline', 'difference_goal_achieved', 'sense_action_str_m', 'sense_action_str_a', 'sense_action_str_t', 'sense_action', 'States', 'unknown', 'Boost_Sense', 'Boost', 'Attendance'], axis=1, inplace = True)

In [ ]:
walkmate.describe()

In [ ]:
test_dataframe = walkmate.loc[walkmate['participant_id'].isin(test_users)]
train_dataframe = walkmate.drop(test_dataframe.index) 
train_dataframe = train_dataframe.drop(train_dataframe.sample(frac=(1-titration_portion)).index)

print(
    "Using %d samples for training and %d for validation"
    % (len(train_dataframe), len(test_dataframe))
)

In [ ]:
print(train_dataframe.columns)

**Step 4: Normalized processing**

In [ ]:
def dataframe_to_dataset(dataframe):
    dataframe = dataframe.copy()
    dataframe.pop("participant_id")
    labels_m = dataframe.pop("sense_action_m")
    labels_a = dataframe.pop("sense_action_a")
    labels_t = dataframe.pop("sense_action_t")
    labels_b = dataframe.pop("boost_action")        
    labels_o = dataframe.pop("other_action")
    labels = pd.concat([labels_m, labels_a, labels_t, labels_b, labels_o], axis = 1)
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    ds = ds.shuffle(buffer_size=len(dataframe))
    return ds


train_ds = dataframe_to_dataset(train_dataframe)
test_ds = dataframe_to_dataset(test_dataframe)

In [ ]:

for x, y in train_ds.take(1):
    print("Input:", x)
    print("Target:", y)

**Step 5: Build the model framework**

In [ ]:
TARGET_FEATURE_LABELS = ["a", "m", "n", "o", "b"]

NUMERIC_FEATURE_NAMES = [
    "study_day_id",
    "block_id",
    "step_count_prev",
    "step_count_prev_1",
    "step_count_prev_2",
    "step_count_prev_3",   
    "step_count_prev_4",
    "age_in_years",
    "baseline_step",
    "goal_steps",
    "conv_turn_avg",
    "duration_avg",
    "week",
    "binary_age",
]

CATEGORICAL_FEATURES_WITH_VOCABULARY = {
    "user_state_m": list(walkmate["user_state_m"].unique()),
    "user_state_a": list(walkmate["user_state_a"].unique()),
    "user_state_t": list(walkmate["user_state_t"].unique()),
    "perceived_state_a": list(walkmate["perceived_state_a"].unique()),
    "perceived_state_m": list(walkmate["perceived_state_m"].unique()),
    "Gender": list(walkmate["Gender"].unique()),
}

CATEGORICAL_FEATURE_NAMES = list(CATEGORICAL_FEATURES_WITH_VOCABULARY.keys())

FEATURE_NAMES = NUMERIC_FEATURE_NAMES + CATEGORICAL_FEATURE_NAMES

#COLUMN_DEFAULTS = [
##    [0] if feature_name in NUMERIC_FEATURE_NAMES + [TARGET_FEATURE_NAME] else ["NA"]
#    for feature_name in CSV_HEADER
#]

NUM_CLASSES = 5
print(len(FEATURE_NAMES))

In [ ]:
from sklearn.metrics import roc_auc_score

def auroc(y_true, y_pred):
    return tf.numpy_function(roc_auc_score, (y_true, y_pred), tf.double)

In [ ]:
learning_rate = 0.1
dropout_rate = 0.01
batch_size = 64
num_epochs = 30


hidden_units = [16, 16]   

def custom_loss(y_true, y_pred):
    loss_m = tf.keras.losses.binary_crossentropy(y_true[:,0], y_pred[:,0], from_logits=False)
    loss_a = tf.keras.losses.binary_crossentropy(y_true[:,1], y_pred[:,1], from_logits=False)
    loss_t = tf.keras.losses.binary_crossentropy(y_true[:,2], y_pred[:,2], from_logits=False)
    loss_b = tf.keras.losses.binary_crossentropy(y_true[:,3], y_pred[:,3], from_logits=False)
    loss_o = tf.keras.losses.binary_crossentropy(y_true[:,4], y_pred[:,4], from_logits=False)
    sense_loss = tf.add(loss_m, loss_a, loss_t)
    non_sense_loss = tf.add(loss_b, loss_o)
    return tf.add(sense_loss, non_sense_loss)


def my_multi_label_metric_fn(y_true, y_pred):
    y_pred_binary = tf.math.greater(y_pred, tf.constant([0.5]))
    y_true_binary = tf.dtypes.cast(y_true, tf.bool)
    action_accuracy = tf.math.reduce_any(tf.math.logical_and(y_true_binary, y_pred_binary), axis=1)
    non_action_accuracy = tf.math.reduce_all(tf.math.logical_or(tf.math.logical_and(tf.math.logical_not(y_true_binary), tf.math.logical_not(y_pred_binary)), y_true_binary), axis=1)
    weights = tf.divide(tf.cast(tf.math.count_nonzero(y_true, axis=1), tf.float32), tf.constant(3.0))
    weighted_accuracy = tf.add(tf.math.multiply(tf.cast(action_accuracy, tf.float32), tf.cast(weights, tf.float32)), tf.math.multiply(tf.cast(non_action_accuracy, tf.float32),  tf.math.subtract(tf.constant(1.0), tf.cast(weights, tf.float32))))
    return tf.reduce_mean(weighted_accuracy)  # Note the `axis=-1`


def my_metric_fn(y_true, y_pred):
    m = tf.keras.metrics.BinaryAccuracy()
    m.update_state(y_true, y_pred)
    return tf.reduce_mean(m.result())  # Note the `axis=-1`


def my_metric_motivation_fn(y_true, y_pred):
    m_m = tf.keras.metrics.BinaryAccuracy()
    m_m.update_state(y_true[:, 0], y_pred[:, 0])  
    return tf.reduce_mean(m_m.result())  # Note the `axis=-1`


def my_metric_ability_fn(y_true, y_pred):
    m_a = tf.keras.metrics.BinaryAccuracy()
    m_a.update_state(y_true[:, 1], y_pred[:, 1])
    return tf.reduce_mean(m_a.result())


def my_metric_trigger_fn(y_true, y_pred):
    m_p = tf.keras.metrics.BinaryAccuracy()
    m_p.update_state(y_true[:, 2], y_pred[:, 2])    
    return tf.reduce_mean(m_p.result()) 


def my_metric_boost_fn(y_true, y_pred):
    m_p = tf.keras.metrics.BinaryAccuracy()
    m_p.update_state(y_true[:, 3], y_pred[:, 3])    
    return tf.reduce_mean(m_p.result()) 


def my_metric_other_fn(y_true, y_pred):
    m_p = tf.keras.metrics.BinaryAccuracy()
    m_p.update_state(y_true[:, 4], y_pred[:, 4])    
    return tf.reduce_mean(m_p.result()) 
                          
    
def run_experiment(model):        
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss=custom_loss,
        metrics=[my_metric_fn, my_metric_motivation_fn, my_metric_ability_fn, my_metric_trigger_fn, my_metric_boost_fn, my_metric_other_fn, tf.keras.metrics.AUC(multi_label=True, num_labels=NUM_CLASSES)],
        run_eagerly=True,
    )
    train_dataset = train_ds.batch(batch_size)
    test_dataset = test_ds.batch(batch_size)

    print("Start training the model...")
    history = model.fit(train_dataset, epochs=num_epochs)
    print("Model training finished")

    #_, train_accuracy = model.evaluate(test_dataset, verbose=0)
    _, accuracy, mm, ma, mp, mb, mo, auc = model.evaluate(test_dataset, verbose=0)

    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    print("Motivation : %f Ability : %f Trigger : %f Boost : %f Others : %f",  mm, ma, mp, mb, mo)
    print("auc ", auc)
#    print(f"Test accuracy: {round(mlc_accuracy * 100, 2)}%")

In [ ]:
def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        if feature_name in NUMERIC_FEATURE_NAMES:
            inputs[feature_name] = layers.Input(
                name=feature_name, shape=(), dtype=tf.float32
            )
        else:
            inputs[feature_name] = layers.Input(
                name=feature_name, shape=(), dtype=tf.string
            )
    return inputs

In [ ]:
from tensorflow.keras.layers import StringLookup


def encode_inputs(inputs, use_embedding=False):
    encoded_features = []
    for feature_name in inputs:
        if feature_name in CATEGORICAL_FEATURE_NAMES:
            vocabulary = CATEGORICAL_FEATURES_WITH_VOCABULARY[feature_name]
            # Create a lookup to convert string values to an integer indices.
            # Since we are not using a mask token nor expecting any out of vocabulary
            # (oov) token, we set mask_token to None and  num_oov_indices to 0.
            lookup = StringLookup(
                vocabulary=vocabulary,
                mask_token=None,
                num_oov_indices=0,
                output_mode="int" if use_embedding else "binary",
            )
            if use_embedding:
                # Convert the string input values into integer indices.
                encoded_feature = lookup(inputs[feature_name])
                embedding_dims = int(math.sqrt(len(vocabulary)))
                # Create an embedding layer with the specified dimensions.
                embedding = layers.Embedding(
                    input_dim=len(vocabulary), output_dim=embedding_dims
                )
                # Convert the index values to embedding representations.
                encoded_feature = embedding(encoded_feature)
            else:
                # Convert the string input values into a one hot encoding.
                encoded_feature = lookup(tf.expand_dims(inputs[feature_name], -1))
        else:
            # Use the numerical features as-is.
            encoded_feature = tf.expand_dims(inputs[feature_name], -1)

        encoded_features.append(encoded_feature)

    all_features = layers.concatenate(encoded_features)
    return all_features

In [ ]:
from tensorflow.keras import layers

def create_baseline_lr_model():
    inputs = create_model_inputs()
    features = encode_inputs(inputs)

    outputs = layers.Dense(units=NUM_CLASSES, activation="sigmoid")(features)    
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


baseline_lr_model = create_baseline_lr_model()
keras.utils.plot_model(baseline_lr_model, show_shapes=True, rankdir="LR")
run_experiment(baseline_lr_model)

In [ ]:
for layer in baseline_lr_model.layers:
    print(layer.weights)

In [ ]:
from tensorflow.keras import layers

def create_baseline_model():
    inputs = create_model_inputs()
    features = encode_inputs(inputs)

    for units in hidden_units:
        features = layers.Dense(units)(features)
        features = layers.BatchNormalization()(features)
        features = layers.ReLU()(features)
        features = layers.Dropout(dropout_rate)(features)

    outputs = layers.Dense(units=NUM_CLASSES, activation="sigmoid")(features)    
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


baseline_model = create_baseline_model()
#keras.utils.plot_model(baseline_model, show_shapes=True, rankdir="LR")

In [ ]:
run_experiment(baseline_model)

In [ ]:
ds_test_batch = test_ds.batch(32)
pred_array = np.empty((0, 5), np.float32)
truth_array = np.empty((0, 5), np.float32)

for (x, y) in ds_test_batch:
    prediction = wide_and_deep_model.predict(x)
    pred_array = np.append(pred_array, prediction, axis=0)
    truth_array = np.append(truth_array, y, axis=0)

y_pred_keras = wide_and_deep_model.predict(test_dataset, batch_size=300)
print(wide_and_deep_model.evaluate(test_dataset))
fpr_keras_m, tpr_keras_m, thresholds_keras_m = roc_curve(truth_array[:,0], pred_array[:,0])
auc_keras_m = auc(fpr_keras_m , tpr_keras_m )
fpr_keras_a, tpr_keras_a, thresholds_keras_a = roc_curve(truth_array[:,1], pred_array[:,1]) 
auc_keras_a = auc(fpr_keras_a  , tpr_keras_a)
fpr_keras_t, tpr_keras_t, thresholds_keras_t = roc_curve(truth_array[:,2], pred_array[:,2])
auc_keras_t = auc(fpr_keras_t , tpr_keras_t)
fpr_keras_b, tpr_keras_b, thresholds_keras_b = roc_curve(truth_array[:,3], pred_array[:,3])
auc_keras_b = auc(fpr_keras_b, tpr_keras_b)
fpr_keras_o, tpr_keras_o, thresholds_keras_o = roc_curve(truth_array[:,4], pred_array[:,4])
auc_keras_o = auc(fpr_keras_o , tpr_keras_o )

In [ ]:
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras_m, tpr_keras_m, label='Sense M (area = {:.3f})'.format(auc_keras_m))
plt.plot(fpr_keras_a, tpr_keras_a, label='Sense A (area = {:.3f})'.format(auc_keras_a))
plt.plot(fpr_keras_t, tpr_keras_t, label='Sense P (area = {:.3f})'.format(auc_keras_t))
plt.plot(fpr_keras_b, tpr_keras_b, label='Boost (area = {:.3f})'.format(auc_keras_b))
plt.plot(fpr_keras_o, tpr_keras_o, label='Others (area = {:.3f})'.format(auc_keras_o))

plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve for sense action')
plt.legend(loc='best')
plt.savefig('direct_sense_coaching.pdf')
plt.show()

In [ ]:
plt.savefig('direct_coaching.png')

In [ ]:
import seaborn as sns
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
sns.lineplot(fpr_keras_m, tpr_keras_m, label='Motivation (area = {:.3f})'.format(auc_keras_m))
# plt.plot(fpr_keras_a, tpr_keras_a, label='Ability (area = {:.3f})'.format(auc_keras_a))
# plt.plot(fpr_keras_t, tpr_keras_t, label='Propensity (area = {:.3f})'.format(auc_keras_t))

plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

In [ ]:
def create_wide_and_deep_model():

    inputs = create_model_inputs()
    wide = encode_inputs(inputs)
    wide = layers.BatchNormalization()(wide)

    deep = encode_inputs(inputs, use_embedding=True)
    for units in hidden_units:
        deep = layers.Dense(units)(deep)
        deep = layers.BatchNormalization()(deep)
        deep = layers.ReLU()(deep)
        deep = layers.Dropout(dropout_rate)(deep)

    merged = layers.concatenate([wide, deep])
    outputs = layers.Dense(units=NUM_CLASSES, activation="softmax")(merged)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


wide_and_deep_model = create_wide_and_deep_model()
#keras.utils.plot_model(wide_and_deep_model, show_shapes=True, rankdir="LR")

In [ ]:
run_experiment(wide_and_deep_model)

In [ ]:
def create_deep_and_cross_model():

    inputs = create_model_inputs()
    x0 = encode_inputs(inputs, use_embedding=True)

    cross = x0
    for _ in hidden_units:
        units = cross.shape[-1]
        x = layers.Dense(units)(cross)
        cross = x0 * x + cross
    cross = layers.BatchNormalization()(cross)

    deep = x0
    for units in hidden_units:
        deep = layers.Dense(units)(deep)
        deep = layers.BatchNormalization()(deep)
        deep = layers.ReLU()(deep)
        deep = layers.Dropout(dropout_rate)(deep)

    merged = layers.concatenate([cross, deep])
    outputs = layers.Dense(units=NUM_CLASSES, activation="softmax")(merged)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


deep_and_cross_model = create_deep_and_cross_model()
#keras.utils.plot_model(deep_and_cross_model, show_shapes=True, rankdir="LR")

In [ ]:
run_experiment(deep_and_cross_model)

In [ ]:
GROWING_STRATEGY = "BEST_FIRST_GLOBAL"
NUM_TREES = 250
MIN_EXAMPLES = 6
MAX_DEPTH = 5
SUBSAMPLE = 0.65
SAMPLING_METHOD = "RANDOM"
VALIDATION_RATIO = 0.1

In [ ]:
pip install -U tensorflow_decision_forests

In [ ]:
import tensorflow_decision_forests as tfdf


In [ ]:
def specify_feature_usages(inputs):
    feature_usages = []

    for feature_name in inputs:
        if inputs[feature_name].dtype == tf.dtypes.float32:
            feature_usage = tfdf.keras.FeatureUsage(
                name=feature_name, semantic=tfdf.keras.FeatureSemantic.NUMERICAL
            )
        else:
            feature_usage = tfdf.keras.FeatureUsage(
                name=feature_name, semantic=tfdf.keras.FeatureSemantic.CATEGORICAL
            )

        feature_usages.append(feature_usage)
    return feature_usages

In [ ]:
def create_gbt_model():
    gbt_model = tfdf.keras.GradientBoostedTreesModel(
        features=specify_feature_usages(create_model_inputs()),
        exclude_non_specified_features=True,
        growing_strategy=GROWING_STRATEGY,
        num_trees=NUM_TREES,
        max_depth=MAX_DEPTH,
        min_examples=MIN_EXAMPLES,
        subsample=SUBSAMPLE,
        validation_ratio=VALIDATION_RATIO,
        task=tfdf.keras.Task.CLASSIFICATION,
        loss="DEFAULT",
    )

    gbt_model.compile(metrics=[keras.metrics.BinaryAccuracy(name="accuracy")])
    return gbt_model

In [ ]:
def prepare_sample(features, target):
    for feature_name in features:
        if feature_name in CATEGORICAL_FEATURES_WITH_VOCABULARY:
            if features[feature_name].dtype != tf.dtypes.string:
                # Convert categorical feature values to string.
                features[feature_name] = tf.strings.as_string(features[feature_name])
    return features, target


def run_experiment(model, train_data, test_data, num_epochs=1, batch_size=None):
    train_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(
        train_data, label='block_action_m'
    ).map(prepare_sample, num_parallel_calls=tf.data.AUTOTUNE)
    test_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(
        test_data, label='block_action_m'
    ).map(prepare_sample, num_parallel_calls=tf.data.AUTOTUNE)    
    #train_tfdf = train_data.map(prepare_sample, num_parallel_calls=tf.data.AUTOTUNE)
    #test_tfdf = test_data.map(prepare_sample, num_parallel_calls=tf.data.AUTOTUNE)
    
    model.fit(train_dataset, epochs=num_epochs, batch_size=batch_size,  verbose=0)
    _, accuracy = model.evaluate(test_dataset, verbose=2)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

In [ ]:
gbt_model = create_gbt_model()
#keras.utils.plot_model(gbt_model, show_shapes=True, rankdir="LR")

In [ ]:
run_experiment(gbt_model, train_dataframe, test_dataframe)

In [ ]:
print(gbt_model.summary())